# Interactive Assessment Graphs
To make the findings of our module assessment information breakdown easier, we have proposed an interactive graphing tool for ease of use in finding the difference between schools, level and modules. This Jupyter Notebook is for testing and development of said tool.

## Imports
We will begin by importing necessary packages and helper functions for finding the input files.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import re
import pathlib as Path
import html5lib
import json
import plotly.graph_objects as go
import ipywidgets 

The below function is for finding module assessment information files saved by the Module Scraper Code. It is a helper function to make our lives easier.

In [2]:
#This finds files that were saved by the Module Scraper Code
def file_finder(school=None, level=None, givenList=None, filename=None, module=None):
    #Set the path to the overall directory with our input files
    dir_raw=Path.Path("ModuleInformation")
    
    #This locates the subdirectory where files by school, level or list can be found
    subdirectory=""
    #Set the subdirectory 
    if level != None:
        subdirectory+= "Level=%d" %(level)
    if school != None:
        subdirectory+= "_School="+school.replace(" ", "-")
    if filename != None:
        subdirectory = filename
    if module != None:
        subdirectory = ("IndividualModules/%s" %module)
        
    #If a subdirectory has indeed been set, change the path to reflect this
    if len(subdirectory) > 0:
        dir_raw =dir_raw / subdirectory
        

    #Read the files for the assessment and descriptors in, closing the file immediately after
    with open(dir_raw / "assessments.json", 'r') as infile:
        print("Reading from %s" % dir_raw)
        if module != None:
            assessments=pd.read_json(infile, orient="columns")
        else:
            assessments=pd.read_json(infile)
    with open(dir_raw / "descriptors.json", 'r') as infile:
        print("Reading from %s" % dir_raw)
        if module != None:
            descriptors=pd.read_json(infile, typ="series")
        else:
            descriptors=pd.read_json(infile)

    #Return the desired assessment and description information
    return assessments, descriptors

Just to check that the above function works, we will run it.

In [3]:
work_type={"Assignment" :"At home", \
                "Attendance": "In person", \
                "Class Test" : "In person", \
                "Continuous Assessment": "At home", \
               "Essay": "At home", \
                "Examination": "In person", \
                "Fieldwork": "In person", \
                "Group Project": "Hybrid", \
                "Journal": "Hybrid",\
               "Lab Report": "Hybrid", \
                "Multiple Choice Questionnaire": "Hybrid", \
                "Oral Examination": "In person", \
               "Portfolio" : "Hybrid",  \
                "Practical Examination": "In person", \
                "Presentation" : "In person", \
                "Project": "At home", \
               "Seminar": "In person", \
               "Studio Examination" : "In person",\
               "Assessments worth <2%": "Unknown"}

In [4]:
#Test the file finder function 
assessments, descriptors = file_finder()

print(len(assessments))
assessments["Level"]=assessments["Level"].apply(lambda x: pd.to_numeric(x.split('(')[0], errors='ignore'))

#This is the dataset cleaning, so that they are ready for presentation
assessments["% of Final Grade"]=pd.to_numeric(assessments["% of Final Grade"], errors='coerce')
print(len(assessments))
assessments=assessments.replace("Multiple Choice Questionnaire (Short)", "Multiple Choice Questionnaire")
print(len(assessments))
assessments["Assessment Type"]=assessments["Assessment Type"].astype("category")
print(len(assessments))
    
    
assessments

#Builds a graph to indicate the susceptibility of a module set's assessments to Chat GPT
def workType_pie(Weighted=True, School="All", Level=0, module=""):
    new_assessments=assessments
    #First we will need to filter the dataset as desired
    if School != "All":
        new_assessments=assessments[assessments["School"]==School]
    if Level != 0:
        new_assessments=assessments[assessments["Level"]==Level]
    if len(module) > 0:
        new_assessments=assessments[assessments["Module Code"]==module]

    if len(new_assessments)==0:
        print("ERROR: No modules found with given parameters")
        
    num_unique=len(new_assessments["Module Code"].unique())
    
    #Data Cleaning and Manipulation Part
    #########################################################################################
    #First we will reduce the assessments dataframe to just the columns we require here.
    new_assessments=new_assessments[["Assessment Type", "% of Final Grade", "Scaled % of Final Grade", "Work Type"]]

    #For the pie chart, group by susceptibilty
    assessment_sus=new_assessments.groupby(by="Work Type").sum()
    #For the bar chart, group by assessment type
    assessment_total=new_assessments.groupby(by="Assessment Type").sum()

    #Choose whether or not you want to scale by credits, or take all modules to be equal, regardless of credits
    if Weighted==True:
        assessment_scale="Scaled % of Final Grade"
    else:
        assessment_scale="% of Final Grade"
        
    total=assessment_sus[assessment_scale].sum()

    #Get the percentage each assessment type makes up of the overall for both graphs
    assessment_sus["% of Assessment"]=assessment_sus[assessment_scale].apply(lambda x: x/total)
    assessment_total["% of Assessment"]=assessment_total[assessment_scale].apply(lambda x: x/total)

    #The groupby function removed the categorical "Susceptibility" column. Bring it back
    assessment_total["Work Type"]=assessment_total.index.map(lambda x: work_type[x])

    #Sort the totals into descending order, to make it easier to see
    sorted_totals=assessment_total.sort_values(by="% of Assessment", ascending=False)
    #The groupby function removed the categorical "Susceptibility" column. Bring it back
    sorted_totals["Work Type"]=sorted_totals.index.map(lambda x: work_type[x])

    #Graph building part
    #############################################################################################################
    #Make the figure
    fig, (ax, bx)=plt.subplots(ncols=2, figsize=(16, 9))

    #These are for selecting colours and hatching for each pie segment
    colour_dict={"At home":"crimson", "Hybrid": "gold", "In person":"lightgrey"}
    hatching_dict={"At home": None, "Hybrid": None, "In person":"/"}
    colours=[]
    hatching=[]
    #Get the colours and hatching for each pie segment
    for segment in assessment_sus.index:
        colours.append(colour_dict[segment])
        hatching.append(hatching_dict[segment])

    #Make the susceptibility pie chart
    ax.pie(assessment_sus["% of Assessment"], wedgeprops = { 'linewidth' : 1.5, 'edgecolor' : 'white' }, \
           autopct="%1.1f%%", labels=assessment_sus.index, hatch=hatching ,\
           colors=colours, textprops={'fontsize':18})

    #Set the title of the pie chart for ease of understanding
    ax.set_title("Proportions of Different Assessment Kinds", fontsize=20)
    ax.set_xlabel("\n\nNumber of Modules in this Breakdown: %d" %(num_unique), fontsize=20)
    #These are for selecting colours and hatching for each bar in the bar chart
    colour_dict={"At home":"crimson", "Hybrid": "gold", "In person":"lightgrey"}
    hatching_dict={"At home": None, "Hybrid": None, "In person":"/"}
    colours=[]
    hatching=[]
    #Get the colours and hatching type based on susceptibilty
    for sus in sorted_totals["Work Type"]:
        colours.append(colour_dict[sus])
        hatching.append(hatching_dict[sus])
        
        
    #Make the assessment type breakdown bar chart, with colour and hatching indicating the susceptibility
    bx.barh(sorted_totals.index, sorted_totals["% of Assessment"].apply(lambda x:x*100), edgecolor="white", hatch = hatching ,\
           color=colours)
    
    #Set the options for the axes
    bx.tick_params(axis='y', labelsize=16)
    bx.tick_params(axis='x', labelsize=14)
    #ax.legend(fontsize=20)
    bx.set_xlabel("% of Assessment",fontsize=18)
    #Set the title for the bar chart for ease of understanding
    bx.set_title("Assessment Type Breakdown", fontsize=20)
    

    #Here we want to set the overall title of the charts - to indicate what module subset they are describing
    what="College of Engineering and Architecture"
    if School != 'All':
        what=""
        what+="School of "
        what+=School
    if Level != 0:
        what+= " At Level %d" %Level
    #if filename != None:
    #    what="".join([(" "+i if i.isupper() else i) for i in filename])
    if len(module) > 0:
        what = module + " Module"
        
    #Set the overall title for the two graphs
    fig.suptitle("Assessment Breakdown in the %s" %what, fontsize=24)
    fig.tight_layout()

Reading from ModuleInformation
Reading from ModuleInformation
1272
1272
1272
1272


In [5]:
school_list=assessments["School"].unique().tolist()
school_list.append("All")

moduleGetter=ipywidgets.Combobox(
    # value='John',
    placeholder='Enter Module Code',
    options=assessments["Module Code"].unique().tolist(),
    description='Module:',
    ensure_option=True,
)

ipywidgets.interact_manual(workType_pie, School=school_list, Level=(0, 5, 1), module=moduleGetter)

C:\Users\catha\AppData\Local\Temp\ipykernel_12208\1196306440.py:12: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  ipywidgets.interact_manual(workType_pie, School=school_list, Level=(0, 5, 1), module=moduleGetter)


interactive(children=(Checkbox(value=True, description='Weighted'), Dropdown(description='School', index=6, op…

<function __main__.workType_pie(Weighted=True, School='All', Level=0, module='')>